In [1]:
import os

In [2]:
trained_models_dir = 'trained_models_dir'
trained_models_dir = trained_models_dir + '/lgb'
if not os.path.exists(trained_models_dir):
    os.makedirs(trained_models_dir)

In [3]:
search_results = []
search_results_file = os.path.join(trained_models_dir, 'search_results.json')

In [4]:
import json

In [5]:
with open(search_results_file, 'r') as file:
    search_results = json.load(file)

In [6]:
val_scores = [record['scores']['val'] for record in search_results]
sorted_scores = sorted(val_scores)

In [7]:
import pandas as pd

In [8]:
_train_data = pd.read_pickle('D:/ml_data/pd_data_train.pkl')
_valid_data = pd.read_pickle('D:/ml_data/pd_data_valid.pkl')
_test_data = pd.read_pickle('D:/ml_data/pd_data_test.pkl')

In [9]:
def get_X_y(data):
    #     data = data[data.date_block_num > 11]
    X = data.drop(['date_block_num', 'item_cnt_month', 'shop_id', 'item_id'], axis=1)
    y = data['item_cnt_month']
    return X, y

In [10]:
train_X, train_y = get_X_y(_train_data)
valid_X, valid_y = get_X_y(_valid_data)
test_X, test_y = get_X_y(_test_data)

del _train_data
del _valid_data
del _test_data

In [11]:
from tqdm.notebook import tqdm

In [12]:
import lightgbm as lgb

In [13]:
train_preds = []
valid_preds = []
test_preds = []

for i in tqdm(range(15)):
    min_value = sorted_scores[i]
    min_index = val_scores.index(min_value)
    
    best_model = search_results[min_index]
    
    bst = lgb.Booster(model_file=os.path.join(best_model['model_path'], 'lgb_model_filtered.txt'))
    train_pred = bst.predict(train_X)
    valid_pred = bst.predict(valid_X)
    test_pred = bst.predict(test_X)
    
    train_preds.append(train_pred)
    valid_preds.append(valid_pred)
    test_preds.append(test_pred)
    

In [14]:
def reshape_preds(preds):
    for i, pred in enumerate(preds):
        pred = pred.reshape(pred.shape[0], 1)
        preds[i] = pred

    for pred in preds:
        print(pred.shape)
    
    return preds

In [15]:
train_preds = reshape_preds(train_preds)

(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)
(4777260, 1)


In [16]:
valid_preds = reshape_preds(valid_preds)
test_preds = reshape_preds(test_preds)

(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(238172, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)
(214200, 1)


In [17]:
import numpy as np

In [18]:
train_data = np.concatenate(train_preds, axis=1)

In [19]:
valid_data = np.concatenate(valid_preds, axis=1)
test_data = np.concatenate(test_preds, axis=1)

In [20]:
from lightgbm import LGBMRegressor

In [32]:
import xgboost as xgb
# xgb_params = {
#     'seed': 123,
#     'colsample_bytree': 0.7,
#     'silent': 1,
#     'subsample': 0.7,
#     'learning_rate': 0.075,
#     'objective': 'binary:logistic',
#     'max_depth': 4,
#     'num_parallel_tree': 1,
#     'min_child_weight': 1,
#     'nrounds': 200
# }
# model = xgb.XGBRegressor(seed=123)
model = LGBMRegressor(seed=123)

model.fit(
    train_data, 
    train_y, 
    eval_metric="rmse", 
    eval_set=[(train_data, train_y), (valid_data, valid_y)], 
    verbose=True, 
    early_stopping_rounds=10,
)

[1]	training's rmse: 1.38997	training's l2: 1.93202	valid_1's rmse: 1.09678	valid_1's l2: 1.20292
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 1.33279	training's l2: 1.77634	valid_1's rmse: 1.06079	valid_1's l2: 1.12527
[3]	training's rmse: 1.28445	training's l2: 1.64981	valid_1's rmse: 1.03044	valid_1's l2: 1.06181
[4]	training's rmse: 1.24387	training's l2: 1.5472	valid_1's rmse: 1.00585	valid_1's l2: 1.01173
[5]	training's rmse: 1.20982	training's l2: 1.46368	valid_1's rmse: 0.985835	valid_1's l2: 0.971871
[6]	training's rmse: 1.18147	training's l2: 1.39586	valid_1's rmse: 0.969727	valid_1's l2: 0.94037
[7]	training's rmse: 1.1579	training's l2: 1.34073	valid_1's rmse: 0.956659	valid_1's l2: 0.915197
[8]	training's rmse: 1.13826	training's l2: 1.29564	valid_1's rmse: 0.945966	valid_1's l2: 0.894852
[9]	training's rmse: 1.12212	training's l2: 1.25916	valid_1's rmse: 0.937768	valid_1's l2: 0.879409
[10]	training's rmse: 1.1088	training's l2: 1.2294

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, seed=123,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [28]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse_sklearn(y_actual, y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [31]:
rmse_sklearn(train_y, model.predict(train_data))

1.0368996963181198

In [29]:
rmse_sklearn(valid_y, model.predict(valid_data))

0.9006118999007613

In [30]:
rmse_sklearn(test_y, model.predict(test_data))

0.8480768874035476

In [24]:
test_preds = model.predict(test_data)

In [25]:
test_preds

array([0.6402179 , 0.40975285, 1.023922  , ..., 0.07356885, 0.05130315,
       0.06615463], dtype=float32)

In [26]:
from module.data.read_data import *

test = test_file_processing().set_index('ID')
test.shape

(214200, 3)

In [27]:
import pickle

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": test_preds.clip(0, 20),
})

submission.to_csv('ensamble_submission.csv', index=False)
pickle.dump(test_preds, open('ensamble_test.pickle', 'wb'))

In [33]:
from sklearn.linear_model import LinearRegression

In [35]:
logistic_regression = LinearRegression()
logistic_regression.fit(train_data, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [36]:
rmse_sklearn(train_y, logistic_regression.predict(train_data))

1.0289852674014432

In [37]:
rmse_sklearn(valid_y, logistic_regression.predict(valid_data))

0.902103593832498

In [38]:
rmse_sklearn(test_y, logistic_regression.predict(test_data))

0.9156722806371645

In [39]:
test_preds = logistic_regression.predict(test_data)

In [40]:
test_preds

array([ 0.67985803,  0.40878188,  1.20240332, ...,  0.06233808,
       -0.00574399,  0.04421888])

In [41]:
from module.data.read_data import *

test = test_file_processing().set_index('ID')
test.shape

(214200, 3)

In [42]:
import pickle

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": test_preds.clip(0, 20),
})

submission.to_csv('ensamble_submission.csv', index=False)
pickle.dump(test_preds, open('ensamble_test.pickle', 'wb'))